#**합성곱 신경망 구현**

28x28 크기의 흑백 이미지와 3x3 크기의 커널 10개로 합성곱을 수행할 것이다.   
그 다음 2x2 크기의 최대풀링을 수행하여 14x14x10으로 특성 맵의 크기를 줄인다.   
이후 이 특성 맵을 일렬로 펼쳐 100개의 뉴런의 완전연결층과 연결시킨다.   
그 다음 10개의 클래스를 구분하기 위해 소프트맥스 함수를 연결한다.   


In [2]:
# import dependencies

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf


# MultiClassNetwork
# MultiClassNetwork class

class MultiClassNetwork:

  def __init__(self, units=10, batch_size=32, learning_rate=0.1, l1=0, l2=0):
    self.units = units  # 은닉층 개수
    self.w1 = None      # 은닉층 가중치
    self.b1 = None      # 은닉층 절편
    self.w2 = None      # 출력층 가중치
    self.b2 = None      # 출력층 절편
    self.a1 = None      # 은닉층 활성화 출력
    self.losses = []    # 훈련세트 손실 리스트
    self.val_losses = []  # 검증세트 손실 리스트
    self.lr = learning_rate  # 학습률
    self.l1 = l1  # L1 규제값
    self.l2 = l2  # L2 규제값
    self.batch_size = batch_size  # 배치 사이즈


  def init_weights(self, n_features, n_classes):  # 가중치 초기화
    self.w1 = np.random.normal(0, 1, (n_features, self.units))  # (입력 특성 개수, 은닉층 유닛 개수)
    self.b1 = np.zeros(self.units)  # 은닉층 유닛 개수
    self.w2 = np.random.normal(0, 1, (self.units, n_classes)) # (은닉층 유닛 개수, 분류 클래스 개수(출력층 유닛))
    self.b2 = np.zeros(n_classes) # 분류 클래스 개수(출력층 유닛)


  def forpass(self, x):   # 정방향 계산
    z1 = np.dot(x, self.w1) + self.b1   # z1 = XW + B (은닉층 계산)
    self.a1 = self.sigmoid(z1)          # 시그모이드 함수 통과
    z2 = np.dot(self.a1, self.w2) + self.b2   # z2 = AW + B (출력층 계산)
    return z2

  
  def backprop(self, x, err): # 역방향 계산
    m = len(x)
    w2_grad = np.dot(self.a1.T, err) / m  # 출력층 가중치 그레이디언트  A1.T(-(Y - A2)) --> (-(Y - A2)) = err
    b2_grad = np.sum(err) / m     
    err_to_hidden = np.dot(err, self.w2.T) * self.a1 * (1 - self.a1)  # 은닉층 손실함수 미분 오차량 -(Y - A2)W2.T * A1 * (1 - A1)
    w1_grad = np.dot(x.T, err_to_hidden) / m      # 은닉층 가중치 그레이디언트 X.T(err) / m
    b1_grad = np.sum(err_to_hidden, axis=0) / m
    return w1_grad, b1_grad, w2_grad, b2_grad


  def fit(self, x, y, epochs=100, x_val=None, y_val=None):  # 피팅 함수
    np.random.seed(42)  
    self.init_weights(x.shape[1], y.shape[1])   # 가중치 초기화(특성, 분류 클래스 개수)
    for i in range(epochs):
      loss = 0
      print('.', end=' ')
      for x_batch, y_batch in self.gen_batch(x, y):   # 배치만큼 반복
        a = self.training(x_batch, y_batch)
        a = np.clip(a, 1e-10, 1-1e-10)
        loss += np.sum(-y_batch * np.log(a))
      self.losses.append((loss + self.reg_loss()) / len(x))
      self.update_val_loss(x_val, y_val)


  def gen_batch(self, x, y):
    length = len(x)
    bins = length // self.batch_size
    if length % self.batch_size:
      bins += 1
    indexes = np.random.permutation(np.arange(len(x)))
    x = x[indexes]
    y = y[indexes]
    for i in range(bins):
      start = self.batch_size * i
      end = self.batch_size * (i + 1)
      yield x[start:end], y[start:end]


  def sigmoid(self, z):
    z = np.clip(z, -100, None)
    a = 1 / (1 + np.exp(-z))
    return a


  def training(self, x, y):   # 훈련 함수
    m = len(x)  
    z = self.forpass(x)   # 정방향 계산 (출력층 z)
    a = self.softmax(z)   # 소프트맥스  (출력층 결과값 정규화)
    err = -(y - a)        # 크로스엔트로피 손실함수 미분을 위한 오차량 계산
    w1_grad, b1_grad, w2_grad, b2_grad = self.backprop(x, err)  # 역방향 계산
    w1_grad += (self.l1 * np.sign(self.w1) + self.l2 * self.w1) / m   # 은닉층 규제 적용
    w2_grad += (self.l1 * np.sign(self.w2) + self.l2 * self.w2) / m   # 출력층 규제 적용
    self.w1 -= self.lr * w1_grad
    self.b1 -= self.lr * b1_grad
    self.w2 -= self.lr * w2_grad
    self.b2 -= self.lr * b2_grad
    return a


  def softmax(self, z):
    z = np.clip(z, -100, None)
    exp_z = np.exp(z)
    return exp_z / np.sum(exp_z, axis=1).reshape(-1, 1)


  def predict(self, x):
    z = self.forpass(x)
    return np.argmax(z, axis=1)


  def score(self, x, y):
    return np.mean(self.predict(x) == np.argmax(y, axis=1))


  def reg_loss(self):
    return self.l1 * (np.sum(abs(self.w1))) + np.sum(np.abs(self.w2)) + self.l2 / 2 * np.sum(self.w1**2) + np.sum(self.w2**2) # 규제 손실

  
  def update_val_loss(self, x_val, y_val):
    z = self.forpass(x_val)
    a = self.softmax(z)
    a = np.clip(a, 1e-10, 1-1e-10)
    val_loss = np.sum(-y_val * np.log(a))
    self.val_losses.append((val_loss + self.reg_loss()) / len(y_val))

#**구현**

MultiClassNetwork 클래스의 forpass() 메서드에 있던 z1, a1, z2를 계산하는 식은 그대로 두고,   
그 앞에 함성곱과 풀링층을 추가해보자.

###**변경점**

>1. 합성곱층을 통과한 특성 맵 c_out 설정(line:26)
2. 렐루함수를 적용한 특성 맵 r_out 설정(line:27)
3. 풀링층을 통과한 특성 맵 p_out 설정(line:28)
4. 특성 맵을 완전연결층과 연결하기 위해 일렬로 펼친(배치 차원 제외) f_put 설정(line:29)
5. np.dot() 대신 tf.matmul()로 대체 <- conv2d, max_pool2d()가 Tensor 객체를 반환하기 때문
6. sigmoid 대신 relu함수 사용



In [3]:
class ConvolutionNetwork:

  def __init__(self, n_kernels=10, units=10, batch_size=32, learning_rate=0.1, l1=0, l2=0):
    self.n_kernels = n_kernels  # 합성곱 커널 개수
    self.kernel_size = 3  # 커널 크기
    self.optimizer = None # 옵티마이저
    self.conv_w = None
    self.conv_b = None
    self.units = units  # 은닉층 개수
    self.w1 = None      # 은닉층 가중치
    self.b1 = None      # 은닉층 절편
    self.w2 = None      # 출력층 가중치
    self.b2 = None      # 출력층 절편
    self.a1 = None      # 은닉층 활성화 출력
    self.losses = []    # 훈련세트 손실 리스트
    self.val_losses = []  # 검증세트 손실 리스트
    self.lr = learning_rate  # 학습률
    self.batch_size = batch_size  # 배치 사이즈


  def init_weights(self, input_shape, n_classes):  # 가중치 초기화
    g = tf.initializers.glorot_uniform()
    self.conv_w = tf.Variable(g((3, 3, 1, self.n_kernels)))   # 3x3x1xn_kernels 4차원 배열
    self.conv_b = tf.Variable(np.zeros(self.n_kernels), dtype=float)
    n_features = 14 * 14 * self.n_kernels
    self.w1 = tf.Variable(g((n_features, self.units)))  # (입력 특성 개수, 은닉층 유닛 개수)
    self.b1 = tf.Variable(np.zeros(self.units), dtype=float)  # 은닉층 유닛 개수
    self.w2 = tf.Variable(g((self.units, n_classes))) # (은닉층 유닛 개수, 분류 클래스 개수(출력층 유닛))
    self.b2 = tf.Variable(np.zeros(n_classes), dtype=float) # 분류 클래스 개수(출력층 유닛)


  def forpass(self, x):   # 정방향 계산
    c_out = tf.nn.conv2d(x, self.conv_w, strides=1, padding='SAME') + self.conv_b   # 3x3 합성곱 연산
    r_out = tf.nn.relu(c_out)   # 렐루 함수 적용
    p_out = tf.nn.max_pool2d(r_out, ksize=2, strides=1, padding='VALID')    # 2x2 풀링
    f_out = tf.reshape(p_out, [x.shape[0], -1])   # 첫 번째 배치 차원을 제외하고 출력을 일렬로 펼침
    z1 = tf.matmul(f_out, self.w1) + self.b1   # z1 = XW + B (은닉층 계산)  np.dot 대신 tf.matmul 사용
    self.a1 = tf.nn.relu(z1)          # relu 함수 통과
    z2 = tf.matmul(self.a1, self.w2) + self.b2   # z2 = AW + B (출력층 계산)  np.dot -> tf.matmul
    return z2


  def fit(self, x, y, epochs=100, x_val=None, y_val=None):  # 피팅 함수
    np.random.seed(42)  
    self.init_weights(x.shape[1], y.shape[1])   # 가중치 초기화(특성, 분류 클래스 개수)
    self.optimizer = tf.optimizers.SGD(learning_rate=self.lr)   # SGD(확률적 경사 하강법) 옵티마이저
    for i in range(epochs):
      print('에포크', i , end=' ')
      batch_losses = []
      for x_batch, y_batch in self.gen_batch(x, y):   # 배치만큼 반복
        print('.', end='')
        self.training(x_batch, y_batch)
        batch_losses.append(self.get_loss(x_batch, y_batch))
      print()
      self.losses.append(np.mean(batch_losses))
      self.val_losses.append(self.get_loss(x_val, y_val))


  def gen_batch(self, x, y):
    length = len(x)
    bins = length // self.batch_size
    if length % self.batch_size:
      bins += 1
    indexes = np.random.permutation(np.arange(len(x)))
    x = x[indexes]
    y = y[indexes]
    for i in range(bins):
      start = self.batch_size * i
      end = self.batch_size * (i + 1)
      yield x[start:end], y[start:end]


  def training(self, x, y):   # 훈련 함수
    m = len(x)  
    with tf.GradientTape() as tape:
      z = self.forpass(x)   # 정방향 계산 (출력층 z)
      loss = tf.nn.softmax_cross_entropy_with_logits(y, z)    # 손실 계산(각 샘플에 대한 손실)
      loss = tf.reduce_mean(loss)   # 손실의 평균 구하기
    weights_list = [self.conv_w, self.conv_b, self.w1, self.b1, self.w2, self.b2]
    grads = tape.gradient(loss, weights_list)   # 가중치 그레이디언트 계산
    self.optimizer.apply_gradients(zip(grads, weights_list))  # 가중치 업데이트


  def predict(self, x):
    z = self.forpass(x)
    return np.argmax(z, axis=1)


  def score(self, x, y):
    return np.mean(self.predict(x) == np.argmax(y, axis=1))


  def get_loss(self, x, y):
    z = self.forpass(x)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y, z))
    return loss.numpy()


#**합성곱 신경망 훈련하기**

패션 MNIST 데이터셋으로 훈련시켜보자.

In [4]:
(x_train_all, y_train_all), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
x_train, x_val, y_train, y_val = train_test_split(x_train_all, y_train_all, stratify=y_train_all, test_size=0.2, random_state=42)

y_train_encoded = tf.keras.utils.to_categorical(y_train)
y_val_encoded = tf.keras.utils.to_categorical(y_val)

4431872/4422102 [==============================] - 0s 0us/step


In [5]:
x_train = x_train.reshape(-1, 28, 28, 1)
x_val = x_val.reshape(-1, 28, 28, 1)

x_train.shape

(48000, 28, 28, 1)

In [6]:
x_train = x_train / 255
x_val = x_val / 255